In [5]:
# This function will determine Difficulty rating
def difficulty_adjuster(Mastery_metric):
    if Mastery_metric <= (1/3 * 100):
        Difficulty = 1
    elif Mastery_metric <= (2/3  * 100) and Mastery_metric > (1/3  * 100):
        Difficulty = 2
    else:
        Difficulty = 3

    return Difficulty
        
# This function will determine the relevant skill objective given skill and difficulty.
def skill_objective(skill_num, Difficulty):
    skill_objectives_all = [
        ["skill1_dif1", "skill1_dif2", "skill1_dif3"],
        ["skill2_dif1", "skill2_dif2", "skill2_dif3"],
        ["skill3_dif1", "skill3_dif2", "skill3_dif3"],
        # need to update with specific examples
    ]
    
    # Adjusting for zero-based indexing
    return skill_objectives_all[skill_num - 1][Difficulty - 1]

# Example usage
print(difficulty_adjuster(0.5))  # Should return 2
print(skill_objective(1, 2))     # Should return "skill1_dif2"

2
skill1_dif2


In [7]:
# need to add in functionality thats scores max score across attempts. 
def decision(q_score, counter):
    """
    Determines the action to take based on the question score and attempt counter.

    Parameters:
    q_score (int): The score obtained for the question.
    counter (int): The current attempt count for the question.

    Returns:
    int: An integer representing the decision.
         1 - Answer is correct.
         2 - Incorrect and is the 3rd attempt.
         3 - Incorrect but not the 3rd attempt.
    """

    # Decision based on question score and attempt count
    if q_score >= 90:
        action = 1  # Answer correct
    else: 
        if counter < 3:
            counter += 1
            action = 3  # Incorrect and not 3rd Attempt
        else:
            action = 2  # Incorrect and 3rd Attempt

    return action, counter

# Test Cases
print(decision(95, 1))  # Expected: 1 (Answer correct)
print(decision(85, 2))  # Expected: 3 (Incorrect and not 3rd Attempt)
print(decision(80, 3))  # Expected: 2 (Incorrect and 3rd Attempt)

1
3
2


In [11]:
def update_decay_rate(current_decay_rate):
    """
    Update the decay rate by halving it.

    Parameters:
    - current_decay_rate: The current forgetting decay rate.

    Returns:
    - new_decay_rate: The updated decay rate.
    """
    new_decay_rate = current_decay_rate / 2
    return new_decay_rate

import math
def MM_addition(q_score, metric):
    if q_score>=60: # increase values if q_score above 60
        add = math.ceil((100-metric)*(q_score/100)/10) # algorithm to adjust addition depending on mastery score. Round up to deal with larger number. 
    else:
        add = -metric/10 # IF they dont get the answer correct then they loose score, proportional to the metric value.
    
    return add
    

def update_after_q(mastery_metric, retention_metric, current_decay_rate, q_score_max, need_to_revise):
    # Local copies of the metrics
    updated_mastery_metric = mastery_metric
    updated_retention_metric = retention_metric

    if not need_to_revise:
        # Update mastery and retention metrics
        updated_mastery_metric += MM_addition(q_score_max, updated_mastery_metric)
        updated_retention_metric += MM_addition(q_score_max, updated_mastery_metric)
    else: 
        # Update retention metric
        updated_retention_metric += MM_addition(q_score_max, updated_retention_metric)
        
        # Check if the retention metric meets or exceeds the mastery metric
        if updated_retention_metric >= updated_mastery_metric:
            need_to_revise = False
            new_decay_rate = update_decay_rate(current_decay_rate)
            updated_retention_metric = updated_mastery_metric
        else:
            need_to_revise = True
            new_decay_rate = current_decay_rate  # No change in decay rate

    return updated_mastery_metric, updated_retention_metric, need_to_revise, new_decay_rate

In [12]:
def update_learning_status(mastery_metric, retention_metric, forgetting_decay_rate, percentage_forgotten_threshold, need_to_revise):
    """
    Update the learning status based on the given metrics and decay rate.

    Parameters:
    - mastery_metric: A value between 0 and 100 indicating mastery level.
    - retention_metric: A value between 0 and 100 indicating retention level.
    - forgetting_decay_rate: A value between 0 and 1 indicating the rate of forgetting.
    - percentage_forgotten_threshold: A value between 0 and 1 indicating the threshold for forgotten material.

    Returns:
    - need_to_revise: A boolean indicating whether revision is needed.
    - updated_retention_metric: The updated value of the retention metric.
    """
    # Apply decay to the retention metric
    updated_retention_metric = retention_metric * (1 - forgetting_decay_rate)

    # Determine if the retention metric is below the threshold
    if updated_retention_metric < (mastery_metric * percentage_forgotten_threshold):
        need_to_revise = True

    return need_to_revise, updated_retention_metric


In [14]:
def test_update_learning_status():
    test_cases = [
        # Test case 1: No need for revision initially and still no need after update
        {"input": (80, 70, 0.1, 0.5, False), "expected": (False, 63.0)},  # Expected tuple (need_to_revise, updated_retention_metric)
        # Test case 2: Need for revision initially and still need after update
        {"input": (80, 60, 0.2, 0.5, True), "expected": (True, 48.0)},
        # Test case 3: No need for revision initially, but need after update due to decay
        {"input": (80, 65, 0.2, 0.5, False), "expected": (True, 52.0)},
        # Test case 4: Need for revision initially, no longer need after update
        {"input": (80, 75, 0.1, 0.5, True), "expected": (True, 67.5)}
    ]

    for i, test in enumerate(test_cases):
        result = update_learning_status(*test["input"])
        assert result == test["expected"], f"Test case {i+1} failed: expected {test['expected']}, got {result}"

# Run the test function
test_update_learning_status()



AssertionError: Test case 3 failed: expected (True, 52.0), got (False, 52.0)